In [54]:
# The code was removed by Watson Studio for sharing.

In [75]:
#!pip install watson_machine_learning_client==1.0.53

In [78]:
import numpy as np
import pandas as pd

from keras.models import Model, load_model

import h5py as h5
import sys
import types
import zipfile
from project_lib import Project

# https://wml-api-pyclient.mybluemix.net/
from watson_machine_learning_client import WatsonMachineLearningAPIClient



In [56]:
def get_zip(file_name):
    '''
    file_name = Name of zip file you want to download from object storage
    '''
    try:
        fobj = open(file_name, "wb")
        fobj.write(project.get_file(file_name).read()) 
        fobj.close()
        z = zipfile.ZipFile(file_name)
        z.extractall()
    except Exception as e:
        print(Exception,e)
    else:
        print('Files downloaded successfully')
        
get_zip('crime_data.zip')
!ls

<class 'Exception'> Something went wrong! error message:Project___get_file_name_from_asset_metadata(): asset name [crime_data.zip] can not be found!!
a2_m1.json	       a2_m4.json.zip	      my_modelx.h5
a2_m1.json.zip	       a2_m4.json.zip.base64  __pycache__
a2_m1.json.zip.base64  a_yrua4s_anything.log  rklib.py
a2_m2.json	       crime_data_gru.tgz     rklib.pyc
a2_m2.json.zip	       crime_data.h5	      scratch_space
a2_m2.json.zip.base64  crime_data.zip	      sonar.csv
a2_m3.json	       dl4j-snapshot.jar      toronto_part1.csv
a2_m3.json.zip	       model_gru.h5	      toxic.zip
a2_m3.json.zip.base64  model.h5		      watsoniotp.broken.pickle
a2_m4.json	       model.h5.base64	      watsoniotp.healthy.pickle


In [ ]:
## load saved model

In [81]:
model = load_model('model_gru.h5')

with h5.File('crime_data.h5') as f:
    X_train = f['X_train'][...]
    X_test = f['X_test'][...]
    y_train = f['y_train'][...]
    y_test = f['y_test'][...]
    y_pred_proba = f['y_pred'][...]

!tar -cvzf crime_data_gru.tgz model_gru.h5

!ls



model_gru.h5
a2_m1.json	       a2_m4.json.zip	      my_modelx.h5
a2_m1.json.zip	       a2_m4.json.zip.base64  __pycache__
a2_m1.json.zip.base64  a_yrua4s_anything.log  rklib.py
a2_m2.json	       crime_data_gru.tgz     rklib.pyc
a2_m2.json.zip	       crime_data.h5	      scratch_space
a2_m2.json.zip.base64  crime_data.zip	      sonar.csv
a2_m3.json	       dl4j-snapshot.jar      toronto_part1.csv
a2_m3.json.zip	       model_gru.h5	      toxic.zip
a2_m3.json.zip.base64  model.h5		      watsoniotp.broken.pickle
a2_m4.json	       model.h5.base64	      watsoniotp.healthy.pickle


In [58]:
# The code was removed by Watson Studio for sharing.

In [59]:
metadata = {
            wml_client.repository.ModelMetaNames.NAME: 'Crime location prediction Model',
            wml_client.repository.ModelMetaNames.FRAMEWORK_NAME: 'tensorflow',
            wml_client.repository.ModelMetaNames.FRAMEWORK_VERSION: '1.5',
            wml_client.repository.ModelMetaNames.RUNTIME_NAME: 'python',
            wml_client.repository.ModelMetaNames.RUNTIME_VERSION: '3.5',
            wml_client.repository.ModelMetaNames.FRAMEWORK_LIBRARIES: [{'name':'keras', 'version': '2.1.3'}]
}

published_model = wml_client.repository.store_model(model='crime_data_gru.tgz', meta_props=metadata, training_data=X_train, training_target=y_train)

In [60]:
# get my model ide
guid = wml_client.repository.get_model_uid(published_model)
guid

'489c6df9-ae4b-4bc3-8a2e-1b3ea30d934e'

In [61]:
wml_client.repository.list_models()

------------------------------------  -------------------------------  ------------------------  --------------
GUID                                  NAME                             CREATED                   FRAMEWORK
489c6df9-ae4b-4bc3-8a2e-1b3ea30d934e  Crime location prediction Model  2019-01-12T20:07:45.459Z  tensorflow-1.5
------------------------------------  -------------------------------  ------------------------  --------------


In [84]:
## deploy model

In [63]:
deployment = wml_client.deployments.create(guid, 'Crime location prediction Model')

In [79]:
deployment_details = wml_client.deployments.get_details('24d56b17-ddfd-43da-8445-52c27b33b714')

In [ ]:
## scoring URL 

In [80]:
#scoring_url='https://us-south.ml.cloud.ibm.com/v3/wml_instances/5bf45d81-7c64-4339-84e9-afb366ef2db8/deployments/24d56b17-ddfd-43da-8445-52c27b33b714/online'
scoring_url = wml_client.deployments.get_scoring_url(deployment_details)
print(scoring_url)

https://us-south.ml.cloud.ibm.com/v3/wml_instances/5bf45d81-7c64-4339-84e9-afb366ef2db8/deployments/24d56b17-ddfd-43da-8445-52c27b33b714/online


In [ ]:
## test scoring

In [70]:
scoring_data = {'values': [X_test[5].tolist()]}
print (scoring_data)
predictions = wml_client.deployments.score(scoring_url, scoring_data)
print("Scoring result: " + str(predictions))

{'values': [[0.0, 6.0, 24.0, 6.0, 0.0, 0.0]]}
Scoring result: {'fields': ['prediction', 'prediction_classes', 'probability'], 'values': [[[1.5167643141467124e-08, 0.5006483793258667, 0.38024789094924927, 0.11910367012023926], 1, [1.5167643141467124e-08, 0.5006483793258667, 0.38024789094924927, 0.11910367012023926]]]}
